## Computations with (subsystem) stabilizer codes

Overview:
* Representing Pauli operators
* Symplectic Gram-Schmidt procedure
* Minimum distance computation

### Representing Pauli operators

In [4]:
import qiskit_qec.utils.pauli_rep as pauli_rep

# String representation
pauli_string = "iXIXXIIXIIIIIY"

# Convert to a symplectic matrix
matrix, phase_exp = pauli_rep.str2symplectic(pauli_string, qubit_order="left-to-right")
print(matrix.astype(int))

[[1 0 1 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1]]


In [2]:
# Index format, printed left-to-right, index starts at 1 (not 0), with underscores
new_pauli_string = pauli_rep.symplectic2str(matrix, phase_exp,
                                            qubit_order="left-to-right",
                                            index_start=1, index_str="_")
print(new_pauli_string)

iX_1X_3X_4X_7Y_13


In [5]:
from qiskit_qec.operators.base_pauli import BasePauli
from qiskit_qec.operators.pauli import Pauli

# Configure the Pauli object
BasePauli.set_syntax(pauli_rep.INDEX_SYNTAX)
BasePauli.set_qubit_order("left-to-right")

# Create some Pauli operators
str1="-iZ0X1X10Z3Y2"
str2="iXIXIXIXIXI"
str3="Z1Z2X10"
P1 = Pauli(str1)
P2 = Pauli(str2)
P3 = Pauli(str3)
print(P1, P2, P3)

-iZ0X1Y2Z3X10 iX1X3X5X7X9 Z1Z2X10


In [8]:
print(P1.num_qubits)
print(P1.phase)
print(P1.x)
print(P1.z.astype(int))

11
[-0.-1.j]
[False  True  True False False False False False False False  True]
[1 0 1 1 0 0 0 0 0 0 0]


In [9]:
from qiskit_qec.operators.pauli_list import PauliList

# Create a list of Pauli operators
PL = PauliList([P1, P2])
print(PL)

['-iZ0X1Y2Z3X10', 'iX1X3X5X7X9']


In [7]:
# Test commutation of each element with P1
print(PL.all_commutes(P1))

[[ True]
 [False]]


In [8]:
# Left multiply every element by P3 = Z1Z2X10
print(PL.compose(P3))

['-iZ0Y1X2Z3', '-Y1Z2X3X5X7X9X10']


In [11]:
from qiskit import QuantumCircuit

# Conjugate by a Clifford circuit (bit-wise Hadamard)
qc = QuantumCircuit(P1.num_qubits)
for i in range(P1.num_qubits):
    qc.h(i)

print(PL)
print(PL.evolve(qc))

['-iZ0X1Y2Z3X10', 'iX1X3X5X7X9']
['iX0Z1Y2X3Z10', 'iZ1Z3Z5Z7Z9']


In [13]:
# 1. init from list[str]
pauli_list = PauliList(["II", "ZI", "-iYY"])
print(pauli_list)

# 2. init from Pauli
pauli1 = Pauli("iXI")
print(PauliList(pauli1))

# 3. init from list[Pauli]
pauli2 = Pauli("iZZ")
print(PauliList([pauli1, pauli2]))

# 4. init from np.ndarray
import numpy as np
z = np.array([[True, True], [False, False], [True, True]])
x = np.array([[False, True], [True, False], [True, True]])
phase = np.array([0, 1, 3])
pauli_list = PauliList.from_symplectic(z, x, phase)
print(pauli_list)

['', 'Z1', '-iY0Y1']
['iX1']
['iX1', 'iZ0Z1']
['Z0Y1', '-iX0', 'iY0Y1']


### Symplectic Gram-Schmidt procedure

In [14]:
import qiskit_qec.linear.symplectic as symplectic

pauli_strings = ['XXXX', 'ZZZZ']
matrix, _ = pauli_rep.str2symplectic(pauli_strings, qubit_order="left-to-right")
print(matrix.astype(int))

[[1 1 1 1 0 0 0 0]
 [0 0 0 0 1 1 1 1]]


In [21]:
full = symplectic.basis_for_pauli_group(matrix)
print(full.astype(int))

[[1 1 1 1 0 0 0 0]
 [0 0 0 0 1 1 1 1]
 [1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]]


In [26]:
center, x, z = symplectic.symplectic_gram_schmidt(full)
print(pauli_rep.symplectic2str(center, qubit_order="left-to-right"))
print(pauli_rep.symplectic2str(x, qubit_order="left-to-right"))
print(pauli_rep.symplectic2str(z, qubit_order="left-to-right"))


['X0X1X2X3' 'Z0Z1Z2Z3' 'X2X3' 'X1X3']
['Z0' 'X1X2X3' 'Z1Z3' 'Z2Z3']


In [27]:
# 3x3 Bacon-Shor code
pauli_strings = ["XIIXIIIII", "IXIIXIIII", "IIXIIXIII",
                 "IIIXIIXII", "IIIIXIIXI", "IIIIIXIIX",
                 "ZZIIIIIII", "IZZIIIIII", "IIIZZIIII",
                 "IIIIZZIII", "IIIIIIZZI", "IIIIIIIZZ"]
plist = PauliList(pauli_strings)
print(symplectic.is_stabilizer_group(plist.matrix))

False


In [28]:
center = symplectic.center(plist.matrix)
print(pauli_rep.symplectic2str(center, qubit_order="left-to-right"))

['X3X4X5X6X7X8' 'X0X1X2X3X4X5' 'Z1Z2Z4Z5Z7Z8' 'Z0Z1Z3Z4Z6Z7']


In [31]:
center, xp, zp = symplectic.symplectic_gram_schmidt(plist.matrix)
print("S = ", pauli_rep.symplectic2str(center, qubit_order="left-to-right"))
print("X gauge = ", pauli_rep.symplectic2str(xp, qubit_order="left-to-right"))
print("Z gauge = ", pauli_rep.symplectic2str(zp, qubit_order="left-to-right"))

S =  ['X3X4X5X6X7X8' 'X0X1X2X3X4X5' 'Z1Z2Z4Z5Z7Z8' 'Z0Z1Z3Z4Z6Z7']
X gauge =  ['X5X8' 'X4X5X7X8' 'X2X5' 'X1X2X4X5']
Z gauge =  ['Z7Z8' 'Z6Z7' 'Z4Z5Z7Z8' 'Z3Z4Z6Z7']


In [32]:
centerp, xpp, zpp = symplectic.normalizer(center, xp, zp)
print("X all = ", pauli_rep.symplectic2str(xpp, qubit_order="left-to-right"))
print("Z all = ", pauli_rep.symplectic2str(zpp, qubit_order="left-to-right"))

X all =  ['X6X7X8' 'X1X2X4X5' 'X2X5' 'X4X5X7X8' 'X5X8']
Z all =  ['Z2Z5Z8' 'Z3Z4Z6Z7' 'Z4Z5Z7Z8' 'Z6Z7' 'Z7Z8']


### Minimum distance computation

In [34]:
from qiskit_qec.analysis.properties import minimum_distance

In [35]:
# 3x3 Bacon-Shor code
pauli_strings = ["XIIXIIIII", "IXIIXIIII", "IIXIIXIII",
                 "IIIXIIXII", "IIIIXIIXI", "IIIIIXIIX",
                 "ZZIIIIIII", "IZZIIIIII", "IIIZZIIII",
                 "IIIIZZIII", "IIIIIIZZI", "IIIIIIIZZ"]
plist = PauliList(pauli_strings)
minimum_distance(plist.matrix)

3

In [36]:
# [[17,1,7]] code from codetables.de
paulis = [
    "yzizzzizizizzzizy",
    "ixzziizzzzzziizzx",
    "zzxiiziiziziiziix",
    "zizyzzzziiiizzzzy",
    "iiiixizizzizzizix",
    "zzizzyiiiizziiiiy",
    "iizziiyziziiizizy",
    "iziiizixzzzzzzzzx",
    "zzzzzzzzxiziiizix",
    "ziziiizizyiizziiy",
    "iiiizziiiiyzzizzy",
    "izizzizzizixiiiix",
    "zzzziiiizzzzyzizy",
    "iiziiziziiziixzzx",
    "zziizzzzzziizzxix",
    "zizzzizizizzzizyy",
]
gauge = PauliList(list(map(lambda x: x.upper(), paulis)))

In [38]:
import time

In [24]:
tic = time.perf_counter()
d = minimum_distance(gauge.matrix, max_weight=10, method="enumerate")
toc = time.perf_counter()
print(d)
print(f"{toc - tic:0.3f} seconds")

7
96.730 seconds


In [39]:
tic = time.perf_counter()
d = minimum_distance(gauge.matrix, max_weight=10, method="partition")
toc = time.perf_counter()
print(d)
print(f"{toc - tic:0.3f} seconds")

7
0.215 seconds
